## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [1]:
!pip install pyspark

In [2]:
from pyspark import SparkContext 
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

In [3]:
sc = SparkSession.builder.master("local").appName("sample").getOrCreate()
spark = SQLContext(sc)

In [8]:
 spark.sql('drop database if exists country_club cascade;')
 spark.sql('create database country_club;')
 spark.sql('show databases;' )

DataFrame[namespace: string]

### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [5]:
# File location and type
file_location_bookings = "Bookings.csv"
file_location_facilities = "Facilities.csv"
file_location_members = "Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [6]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: string (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: string (nullable = true)



### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [9]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [10]:
spark.sql('USE country_club;')
spark.sql('REFRESH TABLE bookings;')
spark.sql('REFRESH TABLE facilities;')
spark.sql('REFRESH TABLE members;')
spark.sql('SHOW TABLES;')

DataFrame[database: string, tableName: string, isTemporary: boolean]

### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [11]:
query = spark.sql('select * from bookings limit 3')
query.show()

+------+-----+-----+-------------------+-----+
|bookid|facid|memid|          starttime|slots|
+------+-----+-----+-------------------+-----+
|     0|    3|    1|2012-07-03 11:00:00|    2|
|     1|    4|    1|2012-07-03 08:00:00|    2|
|     2|    6|    0|2012-07-03 18:00:00|    2|
+------+-----+-----+-------------------+-----+



#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [12]:
q1_query = spark.sql('''SELECT name, membercost FROM facilities 
                        WHERE membercost != 0''')
q1_query.show()

+--------------+----------+
|          name|membercost|
+--------------+----------+
|Tennis Court 1|       5.0|
|Tennis Court 2|       5.0|
|Massage Room 1|       9.9|
|Massage Room 2|       9.9|
|  Squash Court|       3.5|
+--------------+----------+



####  Q2: How many facilities do not charge a fee to members?

In [13]:
q2_query1 = spark.sql('SELECT count(*) FROM facilities WHERE membercost = 0')
q2_query1.show()

q2_query2 = spark.sql('SELECT name, membercost FROM facilities WHERE membercost = 0')
q2_query2.show()

+--------+
|count(1)|
+--------+
|       4|
+--------+

+---------------+----------+
|           name|membercost|
+---------------+----------+
|Badminton Court|       0.0|
|   Table Tennis|       0.0|
|  Snooker Table|       0.0|
|     Pool Table|       0.0|
+---------------+----------+



#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [14]:
q3_query = spark.sql('''SELECT facid, name, membercost, monthlymaintenance 
                        FROM facilities WHERE membercost != 0 AND 
                        membercost < (monthlymaintenance * 0.2)''')
q3_query.show()


+-----+--------------+----------+------------------+
|facid|          name|membercost|monthlymaintenance|
+-----+--------------+----------+------------------+
|    0|Tennis Court 1|       5.0|               200|
|    1|Tennis Court 2|       5.0|               200|
|    4|Massage Room 1|       9.9|              3000|
|    5|Massage Room 2|       9.9|              3000|
|    6|  Squash Court|       3.5|                80|
+-----+--------------+----------+------------------+



#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [15]:
q4_query = spark.sql('''SELECT * FROM facilities 
                        WHERE facid in (1, 5)''')
q4_query.show()

+-----+--------------+----------+---------+-------------+------------------+
|facid|          name|membercost|guestcost|initialoutlay|monthlymaintenance|
+-----+--------------+----------+---------+-------------+------------------+
|    1|Tennis Court 2|       5.0|     25.0|         8000|               200|
|    5|Massage Room 2|       9.9|     80.0|         4000|              3000|
+-----+--------------+----------+---------+-------------+------------------+



#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [16]:
q5_query = spark.sql('''SELECT name, monthlymaintenance, 
                        CASE WHEN (monthlymaintenance > 100) 
                              THEN 'expensive' 
                              ELSE 'cheap' END AS price_guage 
                        FROM facilities; ''')
q5_query.show()

+---------------+------------------+-----------+
|           name|monthlymaintenance|price_guage|
+---------------+------------------+-----------+
| Tennis Court 1|               200|  expensive|
| Tennis Court 2|               200|  expensive|
|Badminton Court|                50|      cheap|
|   Table Tennis|                10|      cheap|
| Massage Room 1|              3000|  expensive|
| Massage Room 2|              3000|  expensive|
|   Squash Court|                80|      cheap|
|  Snooker Table|                15|      cheap|
|     Pool Table|                15|      cheap|
+---------------+------------------+-----------+



#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [17]:
q6_query = spark.sql('''SELECT firstname,surname FROM members
                        WHERE memid=(SELECT max(memid) FROM members);''')
q6_query.show()

+---------+-------+
|firstname|surname|
+---------+-------+
|   Darren|  Smith|
+---------+-------+



####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [18]:
q7_query = spark.sql('''SELECT DISTINCT memid, firstname, surname 
                      FROM members WHERE memid IN 
                        (SELECT memid FROM bookings WHERE facid = 0 OR facid = 1) 
                      ORDER BY surname;''')

q7_query.show(members_df.count(), truncate = False)

+-----+---------+--------+
|memid|firstname|surname |
+-----+---------+--------+
|15   |Florence |Bader   |
|16   |Timothy  |Baker   |
|12   |Anne     |Baker   |
|8    |Tim      |Boothe  |
|5    |Gerald   |Butters |
|22   |Joan     |Coplin  |
|36   |Erica    |Crumpet |
|7    |Nancy    |Dare    |
|28   |David    |Farrell |
|13   |Jemima   |Farrell |
|0    |GUEST    |GUEST   |
|20   |Matthew  |Genting |
|35   |John     |Hunt    |
|26   |Douglas  |Jones   |
|11   |David    |Jones   |
|4    |Janice   |Joplette|
|10   |Charles  |Owen    |
|17   |David    |Pinker  |
|30   |Millicent|Purview |
|3    |Tim      |Rownam  |
|27   |Henrietta|Rumney  |
|24   |Ramnaresh|Sarwin  |
|2    |Tracy    |Smith   |
|14   |Jack     |Smith   |
|1    |Darren   |Smith   |
|9    |Ponder   |Stibbons|
|6    |Burton   |Tracy   |
+-----+---------+--------+



#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [19]:
q8_query = spark.sql('''SELECT members.firstname || ' ' || members.surname as fullname, facilities.name as facility,
                        CASE
                            WHEN members.memid = 0 THEN 
                            (facilities.guestcost*bookings.slots)
                            WHEN members.memid != 0 THEN
                            (facilities.membercost*bookings.slots) 
                        END AS cost

           FROM members INNER JOIN bookings ON members.memid = bookings.memid
                        INNER JOIN facilities ON bookings.facid = facilities.facid 

          WHERE (DATE(bookings.starttime) >= '2012-09-14' AND 
                DATE(bookings.starttime) < '2012-09-15') AND (
		            (members.memid = 0 AND bookings.slots*facilities.guestcost >= 30) OR
                 (members.memid != 0 AND bookings.slots*facilities.membercost >= 30)
                 )
          ORDER BY cost DESC;''')
      
q8_query.show()

+--------------+--------------+-----+
|      fullname|      facility| cost|
+--------------+--------------+-----+
|   GUEST GUEST|Massage Room 2|320.0|
|   GUEST GUEST|Massage Room 1|160.0|
|   GUEST GUEST|Massage Room 1|160.0|
|   GUEST GUEST|Massage Room 1|160.0|
|   GUEST GUEST|Tennis Court 2|150.0|
|   GUEST GUEST|Tennis Court 2| 75.0|
|   GUEST GUEST|Tennis Court 1| 75.0|
|   GUEST GUEST|Tennis Court 1| 75.0|
|   GUEST GUEST|  Squash Court| 70.0|
|Jemima Farrell|Massage Room 1| 39.6|
|   GUEST GUEST|  Squash Court| 35.0|
|   GUEST GUEST|  Squash Court| 35.0|
|   David Jones|Tennis Court 2| 30.0|
|    Tim Boothe|Tennis Court 2| 30.0|
+--------------+--------------+-----+



#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [20]:
q9_query = spark.sql('''SELECT fullname, facility, cost FROM (
                        SELECT members.firstname || ' ' || members.surname as fullname, 
                        facilities.name as facility,
                        CASE
                            WHEN members.memid = 0 THEN 
                            (facilities.guestcost*bookings.slots)
                            WHEN members.memid != 0 THEN
                            (facilities.membercost*bookings.slots) 
                        END AS cost

            FROM members INNER JOIN bookings ON members.memid = bookings.memid
                         INNER JOIN facilities ON bookings.facid = facilities.facid 
            WHERE 
               DATE(bookings.starttime) >= '2012-09-14' AND 
               DATE(bookings.starttime) < '2012-09-15'
                            ) AS bookings WHERE cost >= 30 ORDER by cost DESC; ''')
            
        
q9_query.show()

+--------------+--------------+-----+
|      fullname|      facility| cost|
+--------------+--------------+-----+
|   GUEST GUEST|Massage Room 2|320.0|
|   GUEST GUEST|Massage Room 1|160.0|
|   GUEST GUEST|Massage Room 1|160.0|
|   GUEST GUEST|Massage Room 1|160.0|
|   GUEST GUEST|Tennis Court 2|150.0|
|   GUEST GUEST|Tennis Court 2| 75.0|
|   GUEST GUEST|Tennis Court 1| 75.0|
|   GUEST GUEST|Tennis Court 1| 75.0|
|   GUEST GUEST|  Squash Court| 70.0|
|Jemima Farrell|Massage Room 1| 39.6|
|   GUEST GUEST|  Squash Court| 35.0|
|   GUEST GUEST|  Squash Court| 35.0|
|   David Jones|Tennis Court 2| 30.0|
|    Tim Boothe|Tennis Court 2| 30.0|
+--------------+--------------+-----+



#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

#### **Q10 - Answer (Without using a Subquery) :**




In [21]:
q10_query1 = spark.sql('''SELECT facilities.name, 
                          sum(CASE
                               WHEN bookings.memid = 0 THEN 
                                  facilities.guestcost*bookings.slots
                               WHEN bookings.memid != 0 THEN
                                  facilities.membercost*bookings.slots END)
                           AS total_revenue 

                      FROM bookings INNER JOIN facilities 
                      ON bookings.facid = facilities.facid 
                      GROUP BY facilities.name HAVING 
                          sum(CASE 
                                WHEN bookings.memid = 0 THEN 
                                  facilities.guestcost*bookings.slots
                               WHEN bookings.memid != 0 THEN
                                  facilities.membercost*bookings.slots END) < 1000
                      ORDER BY total_revenue;''' )
q10_query1.show()

+-------------+-------------+
|         name|total_revenue|
+-------------+-------------+
| Table Tennis|        180.0|
|Snooker Table|        240.0|
|   Pool Table|        270.0|
+-------------+-------------+



#### **Q10 - Answer (Using a Subquery) :**

In [22]:
q10_query2 = spark.sql(''' SELECT name, total_revenue FROM (
                          SELECT facilities.name, 
                          sum(CASE
                               WHEN bookings.memid = 0 THEN 
                                  facilities.guestcost*bookings.slots
                               WHEN bookings.memid != 0 THEN
                                  facilities.membercost*bookings.slots END)
                           AS total_revenue

                      FROM bookings INNER JOIN facilities 
                      ON bookings.facid = facilities.facid GROUP BY facilities.name
                               ) AS agg WHERE total_revenue < 1000 ORDER BY total_revenue;''') 

q10_query2.show()

+-------------+-------------+
|         name|total_revenue|
+-------------+-------------+
| Table Tennis|        180.0|
|Snooker Table|        240.0|
|   Pool Table|        270.0|
+-------------+-------------+

